## Motywacja

1. chcemy wytrenować prosty i szybki klasyfikator
2. przypominamy oznaczenia
  * $y$ przyjmuje jedną z $K$ wartości $\{c_1, \ldots, c_K\}$
  * $(\mathbf{x}_n, y_n)$ to $n$-ty element zbioru treningowego
  * $x_{n,d}$ to $d$-ta cecha $n$-tego elementu zbioru treningowego
3. model musi umieć przewidywać $p(y\mid\mathbf{x})$
4. pomysł - nauczmy się
  * $p(\mathbf{x}\mid y)$
  * $p(y)$
  
  i skorzystajmy ze wzoru Bayesa

$$p(y\mid\mathbf{x}) = \dfrac{p(\mathbf{x}\mid y)p(y)}{\sum_{k=1}^K p(\mathbf{x}\mid y=c_k)p(y=c_k)}$$


## Uwaga

1. ten klasyfikator __nie będzie uczony bayesowsko__
  * rozkładów $p(\mathbf{x}\mid y)$ oraz $p(y)$ będziemy uczyć się metodą MLE lub MAP
  * wzór Bayesa odpowiada tylko za fragment modelu, mianowicie __predykcję__
2. tym razem $p(y)$ to prior, $p(\mathbf{x}\mid y)$ to likelihood, a $p(y\mid\mathbf{x})$ to posterior
  * te pojęcia zawsze występują w pewnym kontekście
    * prior to zawsze wiedza, którą posiadamy przed wykonaniem wnioskowania bayesowskiego
    * likelihood (evidence) to wiedza, którą zdobywamy w trakcie wnioskowania
    * posterior to wiedza końcowa
  * np. jeśli wnioskujemy o hipotezach $\theta$, to musimy sami zdefiniować prior
  * tutaj wnioskujemy o konkretnym przykładzie testowym
    * prior to wstępna wiedza o jego klasie
    * nie definiujemy sami tej wiedzy - prior wynika ze zbioru treningowego
3. trzeba mieć świadomość, że wzór Bayesa jest bardzo uniwersalny
  * można go stosować w różnych kontekstach, nie tylko przy uczeniu modeli
  * dlatego trzeba dobrze zrozumieć wzory, bo łatwo się pomylić
    * samo zrozumienie "matematyki" nie wystarcza - wzory są proste (mnożenie i dzielenie)
    * trzeba rozumieć __dlaczego__ jedne zmienne losowe opisuje prior, a inne likelihood

## Uczenie $p(y)$

1. $N$ danych treningowych $y_1, \ldots, y_N$
2. $y$ przyjmuje jedną z $K$ wartości $c_1, \ldots, c_K$ - rozkład dyskretny
3. zazwyczaj $K<<N$
4. uczenie metodą MLE działa bardzo dobrze

## Uczenie $p(\mathbf{x}\mid y)$

1. musimy wytrenować $K$ rozkładów $p(\mathbf{x}\mid y=c_k)$
2. dzielimy dane treningowe na $K$ podzbiorów rozłącznych (używając wartości $y_n$)
3. bardzo trudne zadanie
  * $\mathbf{x}$ ma $D$ współrzędnych
  * prosta estymacja gęstości w $D$ wymiarach jest praktycznie niewykonalna, jeśli $D$ wynosi przynajmniej kilkadziesiąt
  * nieformalnie, rozmiar zbioru treningowego powinien być rzędu $N^D$
  * _curse of dimensionality_
  * skomplikowane modele generatywne działają, bo mają bardzo dobry prior

### "Naiwne" założenie

1. zakładamy, że __współrzędne__ $\mathbf{x}$ są __warunkowo niezależne__ pod warunkiem klasy $y$
$$p(\mathbf{x}\mid y)=p(x_1\mid y)\cdot\ldots\cdot p(x_D\mid y)$$
2. z reguły __bardzo nieprawdziwe założenie__
  * w przeciwieństwie np. do I.I.D.
    * niezależność przykładów treningowych vs niezależność cech w ramach klasy
  * ale __mocno upraszcza trenowanie__ modelu
3. teraz uczymy się $DK$ rozkładów $p(x_d\mid y=c_k)$
  * każdy z takich rozkładów opisuje jedną cechę w ramach jednej klasy
  * cechy mają rozkład dyskretny lub ciągły na $\mathbb{R}$ - wymiar 1 zamiast $D$
  * mamy dostatecznie dużo przykładów (rzędu $N/K$), żeby uczyć się metodą MLE lub MAP


## Dwa przykładowe warianty

### Bernoulli Naive Bayes

1. zakładamy, że $p(x_d\mid y=c_k)$ ma rozkład dyskretny binarny
  * każda cecha przyjmuje wartość 0 albo 1
2. ustalmy cechę $d$ i klasę $c_k$
  * niech $M_0$ oznacza liczbę przykładów klasy $c_k$, dla których cecha $d$ wynosi zero
  * analogicznie $M_1$ to liczba jedynek
3. estymator MLE
$$\begin{align}
\widehat{p}(x_d=0\mid y=c_k) &= \dfrac{M_0}{M_0+M_1} \\
\widehat{p}(x_d=1\mid y=c_k) &= \dfrac{M_1}{M_0+M_1}
\end{align}$$
  * może spowodować $\dfrac{0}{0}$ przy obliczaniu posteriora
4. użyjmy jednak estymatora MAP
$$\begin{align}
\widehat{p}(x_d=0\mid y=c_k) &= \dfrac{M_0 + \frac12}{M_0 + M_1 + 1} \\
\widehat{p}(x_d=1\mid y=c_k) &= \dfrac{M_1 + \frac12}{M_0 + M_1 + 1}
\end{align}$$
5. w modelu zapisujemy tylko $\widehat{p}(x_d=1\mid y=c_k)$
  * przy predykcji korzystamy ze wzoru
$$ \widehat{p}(x_d=0\mid y=c_k) = 1 - \widehat{p}(x_d=1\mid y=c_k) $$
6. $KD+K$ parametrów klasyfikatora
  * $KD$ na modelowanie $p(\mathbf{x}\mid y)$
  * $K$ na modelowanie $p(y)$

### Gaussian Naive Bayes

1. zakładamy, że $p(x_d\mid y=c_k)$ ma rozkład normalny
  * każda cecha jest liczbą rzeczywistą
2. uczymy się średniej $\mu$ i wariancji $\sigma^2$
  * korzystamy z estymatora MLE
$$\begin{align}
\widehat\mu &= \dfrac{1}{N_k}\sum_{n:\, y_n=c_k} x_n \\
\widehat{\sigma^2} &= \dfrac{1}{N_k}\sum_{n:\, y_n=c_k} (x_n-\widehat\mu)^2
\end{align}$$
  * $N_k$ to liczba przykładów w klasie $c_k$
  * sumujemy tylko po przykładach z klasy $c_k$
  * warto zabezpieczyć się przed wyestymowaniem $\widehat{\sigma^2}=0$ (dlaczego?)
  $$ \widehat{\sigma^2} = max\{\widehat{\sigma^2}, \epsilon\}$$
    $\epsilon$ równy np. $10^{-6}$
3. $2KD+K$ parametrów klasyfikatora
  * $2KD$ na modelowanie $p(\mathbf{x}\mid y)$
  * $K$ na modelowanie $p(y)$

### Materiały dodatkowe

http://blog.datumbox.com/machine-learning-tutorial-the-naive-bayes-text-classifier/

Wikipedia

## Dyskusja

### Inne warianty

1. możemy zmodyfikować Bernoulliego i dopuścić wieloklasowe cechy
  * może być różna liczba dozwolonych wartości dla różnych $d$
2. możemy w ogóle dla różnych $d$ brać różne rozkłady - np. raz dyskretne, raz ciągłe
3. Multinomial Naive Bayes
  * wektory $\mathbf{x}$ są __zmiennej długości__
  * np. wektor opisuje zdanie, a każda współrzędna to słowo
4. warto używać MAP zamiast MLE
  * w wypadku $p(y)$ raczej nie trzeba
  * w wypadku cech musimy się zabezpieczyć przed dzieleniem przez zero
5. można próbować uczyć się metodą PPD, ale to raczej strata czasu
  * i tak model jest "zepsuty" przez "naiwne założenie"

### Dlaczego nie powinno się tak budować klasyfikatorów
1. $p(y)$ łatwe w estymacji
2. zazwyczaj $p(\mathbf{x}\mid y)$ bardzo trudne
3. znacznie prościej uczyć się od razu $p(y\mid\mathbf{x})$
4. a więc Naive Bayes ma sens tylko wtedy, gdy $p(\mathbf{x}\mid y)$ jest łatwe
  * wtedy mamy prosty i szybki model
  * model jest zrozumiały, interpretowalny przez człowieka